In [1]:
# Import Libraries
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import numpy as np
from scipy.stats import linregress

# Read csv files
athletes_bio_csv = pd.read_csv("https://raw.githubusercontent.com/josephwccheng/olympedia_web_scraping/main/data/Olympic_Athlete_Bio.csv")
athlete_results_csv = pd.read_csv("https://raw.githubusercontent.com/josephwccheng/olympedia_web_scraping/main/data/Olympic_Athlete_Event_Results.csv")
medals_csv = pd.read_csv("Resources/Olympic_Games_Medal_Tally.csv")
countries_csv = pd.read_csv("Resources/Olympics_Country.csv")
olympics_games_csv = pd.read_csv("Resources/Olympics_Games.csv")

In [2]:
# Merge athletes results and bio
bio_results_csv = pd.merge(athlete_results_csv, athletes_bio_csv, on='athlete_id', how='outer')
bio_results_csv.head()

,edition,edition_id,country_noc_x,sport,event,result_id,athlete,athlete_id,pos,medal,isTeamSport,name,sex,born,height,weight,country,country_noc_y,description,special_notes
0,1912 Summer Olympics,6,FRA,Tennis,"Singles, Men",44337,Jean-François Blanchy,1,=17,NaN,False,Jean-François Blanchy,Male,12 December 1886,NaN,NaN,France,FRA,Jean-François Blanchy competed at the 1912 and...,NaN
1,1912 Summer Olympics,6,FRA,Tennis,"Doubles, Men",44412,Jean-François Blanchy,1,DNS,NaN,True,Jean-François Blanchy,Male,12 December 1886,NaN,NaN,France,FRA,Jean-François Blanchy competed at the 1912 and...,NaN
2,1920 Summer Olympics,7,FRA,Tennis,"Singles, Men",44578,Jean-François Blanchy,1,=32,NaN,False,Jean-François Blanchy,Male,12 December 1886,NaN,NaN,France,FRA,Jean-François Blanchy competed at the 1912 and...,NaN
3,1920 Summer Olympics,7,FRA,Tennis,"Doubles, Men",44626,Jean-François Blanchy,1,4,NaN,True,Jean-François Blanchy,Male,12 December 1886,NaN,NaN,France,FRA,Jean-François Blanchy competed at the 1912 and...,NaN
4,1920 Summer Olympics,7,FRA,Tennis,"Doubles, Mixed",44692,Jean-François Blanchy,1,=8,NaN,True,Jean-François Blanchy,Male,12 December 1886,NaN,NaN,France,FRA,Jean-François Blanchy competed at the 1912 and...,NaN


In [3]:
# Filter to only summer athletes
summer_bio_results_csv = bio_results_csv.loc[(bio_results_csv['edition'].str.contains('Summer')), ['edition', 'edition_id', 'country_noc_x', 'sport', 'event', 'athlete', 'athlete_id', 'medal', 'name', 'sex', 'born', 'height', 'weight', 'country', 'country_noc_y']]
summer_bio_results_csv

,edition,edition_id,country_noc_x,sport,event,athlete,athlete_id,medal,name,sex,born,height,weight,country,country_noc_y
0,1912 Summer Olympics,6,FRA,Tennis,"Singles, Men",Jean-François Blanchy,1,NaN,Jean-François Blanchy,Male,12 December 1886,NaN,NaN,France,FRA
1,1912 Summer Olympics,6,FRA,Tennis,"Doubles, Men",Jean-François Blanchy,1,NaN,Jean-François Blanchy,Male,12 December 1886,NaN,NaN,France,FRA
2,1920 Summer Olympics,7,FRA,Tennis,"Singles, Men",Jean-François Blanchy,1,NaN,Jean-François Blanchy,Male,12 December 1886,NaN,NaN,France,FRA
3,1920 Summer Olympics,7,FRA,Tennis,"Doubles, Men",Jean-François Blanchy,1,NaN,Jean-François Blanchy,Male,12 December 1886,NaN,NaN,France,FRA
4,1920 Summer Olympics,7,FRA,Tennis,"Doubles, Mixed",Jean-François Blanchy,1,NaN,Jean-François Blanchy,Male,12 December 1886,NaN,NaN,France,FRA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314902,1900 Summer Olympics,2,GBR,Cricket,"Cricket, Men",E. P. Daniel,3100007,Gold,E. P. Daniel,Male,NaN,NaN,NaN,Great Britain,GBR
314903,1900 Summer Olympics,2,GBR,Cricket,"Cricket, Men",J. M. Willcox,3100008,Gold,J. M. Willcox,Male,NaN,NaN,NaN,Great Britain,GBR
314904,1900 Summer Olympics,2,GBR,Cricket,"Cricket, Men",G. P. Brook,3100009,Gold,G. P. Brook,Male,NaN,NaN,NaN,Great Britain,GBR
314905,1948 Summer Olympics,12,IRL,Athletics,"800 metres, Men",Seamus Kelly,3100010,NaN,Seamus Kelly,Male,NaN,NaN,NaN,Ireland,IRL


In [4]:
# Lenght of merged bio_results_csv
len(summer_bio_results_csv)

252859

In [5]:
# Change born column to make it a date format column
# Copy the born column to original_born
summer_bio_results_csv['original_born'] = summer_bio_results_csv['born']

# Check if the date is only a year entry and enter NaT in the born column
# Define a function to check string length and identify if its only a year
def is_year_only(date_str):
    if isinstance(date_str, str):
        date_parts = date_str.split()
        return len(date_parts) == 1
    else:
        return False

# Apply the function to original_born
summer_bio_results_csv['is_year_date'] = summer_bio_results_csv['original_born'].apply(is_year_only)

# If it's only a year enter NaT in the born column
summer_bio_results_csv.loc[summer_bio_results_csv['is_year_date'], 'born'] = pd.NaT

# Drop the column that checked the length of the string and original_born
summer_bio_results_csv = summer_bio_results_csv.drop(['is_year_date', 'original_born'], axis=1)

# Copy the born column to copy_born
summer_bio_results_csv['copy_born'] = summer_bio_results_csv['born']

# Define a function to reformat all valid born dates to calculate age
def convert_to_datetime(date_str):
    try:
        return pd.to_datetime(date_str)
    except ValueError:
        return pd.NaT

# Apply the function to copy_born
summer_bio_results_csv['born'] = summer_bio_results_csv['copy_born'].apply(convert_to_datetime)

# Drop the column copy_born
summer_bio_results_csv = summer_bio_results_csv.drop('copy_born', axis=1)
summer_bio_results_csv

,edition,edition_id,country_noc_x,sport,event,athlete,athlete_id,medal,name,sex,born,height,weight,country,country_noc_y
0,1912 Summer Olympics,6,FRA,Tennis,"Singles, Men",Jean-François Blanchy,1,NaN,Jean-François Blanchy,Male,1886-12-12,NaN,NaN,France,FRA
1,1912 Summer Olympics,6,FRA,Tennis,"Doubles, Men",Jean-François Blanchy,1,NaN,Jean-François Blanchy,Male,1886-12-12,NaN,NaN,France,FRA
2,1920 Summer Olympics,7,FRA,Tennis,"Singles, Men",Jean-François Blanchy,1,NaN,Jean-François Blanchy,Male,1886-12-12,NaN,NaN,France,FRA
3,1920 Summer Olympics,7,FRA,Tennis,"Doubles, Men",Jean-François Blanchy,1,NaN,Jean-François Blanchy,Male,1886-12-12,NaN,NaN,France,FRA
4,1920 Summer Olympics,7,FRA,Tennis,"Doubles, Mixed",Jean-François Blanchy,1,NaN,Jean-François Blanchy,Male,1886-12-12,NaN,NaN,France,FRA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314902,1900 Summer Olympics,2,GBR,Cricket,"Cricket, Men",E. P. Daniel,3100007,Gold,E. P. Daniel,Male,NaT,NaN,NaN,Great Britain,GBR
314903,1900 Summer Olympics,2,GBR,Cricket,"Cricket, Men",J. M. Willcox,3100008,Gold,J. M. Willcox,Male,NaT,NaN,NaN,Great Britain,GBR
314904,1900 Summer Olympics,2,GBR,Cricket,"Cricket, Men",G. P. Brook,3100009,Gold,G. P. Brook,Male,NaT,NaN,NaN,Great Britain,GBR
314905,1948 Summer Olympics,12,IRL,Athletics,"800 metres, Men",Seamus Kelly,3100010,NaN,Seamus Kelly,Male,NaT,NaN,NaN,Ireland,IRL


In [6]:
# Confirm date type in born column in the summer_bio_results
summer_bio_results_csv.dtypes

edition                  object
edition_id                int64
country_noc_x            object
sport                    object
event                    object
athlete                  object
athlete_id                int64
medal                    object
name                     object
sex                      object
born             datetime64[ns]
height                  float64
weight                   object
country                  object
country_noc_y            object
dtype: object

In [7]:
# Count valid entries per column in the summer_bio_results
summer_bio_results_csv.count()

edition          252859
edition_id       252859
country_noc_x    252859
sport            252859
event            252859
athlete          252859
athlete_id       252859
medal             37281
name             252859
sex              252859
born             243496
height           179719
weight           179719
country          252859
country_noc_y    252859
dtype: int64

In [8]:
# Check and add start and end dates  for summer olympics that are missing based on the competition_date
olympics_games_csv.at[1, 'start_date'] = "14-May"
olympics_games_csv.at[1, 'end_date'] = "28-Oct"
olympics_games_csv.at[2, 'start_date'] = "1-Jul"
olympics_games_csv.at[2, 'end_date'] = "26-Nov"

# Only games held before 2024
olympics_games_csv = olympics_games_csv.loc[(olympics_games_csv['year'] < 2024) & (olympics_games_csv['isHeld'].isnull()), :]

olympics_games_csv.head()

,edition,edition_id,edition_url,year,city,country_flag_url,country_noc,start_date,end_date,competition_date,isHeld
0,1896 Summer Olympics,1,/editions/1,1896,Athina,https://olympedia-flags.s3.eu-central-1.amazon...,GRE,6 April,15 April,6 – 13 April,NaN
1,1900 Summer Olympics,2,/editions/2,1900,Paris,https://olympedia-flags.s3.eu-central-1.amazon...,FRA,14-May,28-Oct,14 May – 28 October,NaN
2,1904 Summer Olympics,3,/editions/3,1904,St. Louis,https://olympedia-flags.s3.eu-central-1.amazon...,USA,1-Jul,26-Nov,1 July – 26 November,NaN
3,1908 Summer Olympics,5,/editions/5,1908,London,https://olympedia-flags.s3.eu-central-1.amazon...,GBR,13 July,25 July,27 April – 31 October,NaN
4,1912 Summer Olympics,6,/editions/6,1912,Stockholm,https://olympedia-flags.s3.eu-central-1.amazon...,SWE,6 July,15 July,5 May – 27 July,NaN


In [9]:
# Filter to only summer olympics
summer_olympics_csv = olympics_games_csv.loc[(olympics_games_csv['edition'].str.contains('Summer')), ['edition', 'edition_id', 'year', 'city', 'country_noc', 'start_date', 'end_date']]
summer_olympics_csv

,edition,edition_id,year,city,country_noc,start_date,end_date
0,1896 Summer Olympics,1,1896,Athina,GRE,6 April,15 April
1,1900 Summer Olympics,2,1900,Paris,FRA,14-May,28-Oct
2,1904 Summer Olympics,3,1904,St. Louis,USA,1-Jul,26-Nov
3,1908 Summer Olympics,5,1908,London,GBR,13 July,25 July
4,1912 Summer Olympics,6,1912,Stockholm,SWE,6 July,15 July
6,1920 Summer Olympics,7,1920,Antwerpen,BEL,14 August,30 August
7,1924 Summer Olympics,8,1924,Paris,FRA,5 July,27 July
8,1928 Summer Olympics,9,1928,Amsterdam,NED,28 July,12 August
9,1932 Summer Olympics,10,1932,Los Angeles,USA,30 July,14 August
10,1936 Summer Olympics,11,1936,Berlin,GER,1 August,16 August


In [10]:
# Merge start_date with year and force the date format
# Convert year int to str
summer_olympics_csv = summer_olympics_csv.astype({'year': str}, errors='raise')

# Join start_date with year
summer_olympics_csv['final_start_date'] = summer_olympics_csv['start_date'] + ' ' + summer_olympics_csv['year']

# Apply date format function
summer_olympics_csv['final_start_date'] = summer_olympics_csv['final_start_date'].apply(convert_to_datetime)

# Drop original start_date column
summer_olympics_csv = summer_olympics_csv.drop('start_date', axis=1)
summer_olympics_csv

,edition,edition_id,year,city,country_noc,end_date,final_start_date
0,1896 Summer Olympics,1,1896,Athina,GRE,15 April,1896-04-06 00:00:00
1,1900 Summer Olympics,2,1900,Paris,FRA,28-Oct,1900-05-14 00:00:00
2,1904 Summer Olympics,3,1904,St. Louis,USA,26-Nov,1904-07-01 00:00:00
3,1908 Summer Olympics,5,1908,London,GBR,25 July,1908-07-13 00:00:00
4,1912 Summer Olympics,6,1912,Stockholm,SWE,15 July,1912-07-06 00:00:00
6,1920 Summer Olympics,7,1920,Antwerpen,BEL,30 August,1920-08-14 00:00:00
7,1924 Summer Olympics,8,1924,Paris,FRA,27 July,1924-07-05 00:00:00
8,1928 Summer Olympics,9,1928,Amsterdam,NED,12 August,1928-07-28 00:00:00
9,1932 Summer Olympics,10,1932,Los Angeles,USA,14 August,1932-07-30 00:00:00
10,1936 Summer Olympics,11,1936,Berlin,GER,16 August,1936-08-01 00:00:00


In [11]:
summer_olympics_csv.dtypes

edition                     object
edition_id                   int64
year                        object
city                        object
country_noc                 object
end_date                    object
final_start_date    datetime64[ns]
dtype: object

In [12]:
# Merge summer athletes df and olympics df
summer_bio_results_olympics_csv = pd.merge(summer_bio_results_csv, summer_olympics_csv, on=['edition_id','edition'], how='outer')
summer_bio_results_olympics_csv.tail()

,edition,edition_id,country_noc_x,sport,event,athlete,athlete_id,medal,name,sex,born,height,weight,country,country_noc_y,year,city,country_noc,end_date,final_start_date
252854,2020 Summer Olympics,61,USA,Taekwondo,"Featherweight, Women",Anastasija Zolotic,2506221,Gold,Anastasija Zolotic,Female,2002-11-23,NaN,NaN,United States,USA,2020,Tokyo,JPN,8 August 2021,2021-07-23 20:20:00
252855,2020 Summer Olympics,61,UZB,Athletics,"Triple Jump, Women",Roksana Khudoyarova,2506229,NaN,Roksana Khudoyarova,Female,2001-01-30,NaN,NaN,Uzbekistan,UZB,2020,Tokyo,JPN,8 August 2021,2021-07-23 20:20:00
252856,2020 Summer Olympics,61,UZB,Rhythmic Gymnastics,"Individual, Women",Yekaterina Fetisova,2506234,NaN,Yekaterina Fetisova,Female,2003-01-03,NaN,NaN,Uzbekistan,UZB,2020,Tokyo,JPN,8 August 2021,2021-07-23 20:20:00
252857,2020 Summer Olympics,61,UZB,Swimming,"50 metres Freestyle, Women",Natalya Kritinina,2506239,NaN,Natalya Kritinina,Female,2001-01-06,176.0,64,Uzbekistan,UZB,2020,Tokyo,JPN,8 August 2021,2021-07-23 20:20:00
252858,2020 Summer Olympics,61,UZB,Weightlifting,"Light-Heavyweight, Women",Kumushkhon Fayzullayeva,2506242,NaN,Kumushkhon Fayzullayeva,Female,2002-01-20,NaN,NaN,Uzbekistan,UZB,2020,Tokyo,JPN,8 August 2021,2021-07-23 20:20:00


In [13]:
# Confirm it's the lame length on this merged df
len(summer_bio_results_olympics_csv)

252859

In [14]:
summer_bio_results_olympics_csv.dtypes

edition                     object
edition_id                   int64
country_noc_x               object
sport                       object
event                       object
athlete                     object
athlete_id                   int64
medal                       object
name                        object
sex                         object
born                datetime64[ns]
height                     float64
weight                      object
country                     object
country_noc_y               object
year                        object
city                        object
country_noc                 object
end_date                    object
final_start_date    datetime64[ns]
dtype: object

In [23]:
# Add athlete age column
summer_bio_results_olympics_csv['athlete_age'] = round((summer_bio_results_olympics_csv['final_start_date'] - summer_bio_results_olympics_csv['born']).dt.days / 365,0)
summer_bio_results_olympics_csv

,edition,edition_id,country_noc_x,sport,event,athlete,athlete_id,medal,name,sex,...,height,weight,country,country_noc_y,year,city,country_noc,end_date,final_start_date,athlete_age
0,1896 Summer Olympics,1,FRA,Tennis,"Singles, Men",J. Defert,13,NaN,J. Defert,Male,...,NaN,NaN,France,FRA,1896,Athina,GRE,15 April,1896-04-06 00:00:00,NaN
1,1896 Summer Olympics,1,FRA,Tennis,"Doubles, Men",J. Defert,13,NaN,J. Defert,Male,...,NaN,NaN,France,FRA,1896,Athina,GRE,15 April,1896-04-06 00:00:00,NaN
2,1896 Summer Olympics,1,GER,Tennis,"Doubles, Men",John Pius Boland,79,Gold,John Pius Boland,Male,...,NaN,NaN,Great Britain,GBR,1896,Athina,GRE,15 April,1896-04-06 00:00:00,26.0
3,1896 Summer Olympics,1,GBR,Tennis,"Singles, Men",John Pius Boland,79,Gold,John Pius Boland,Male,...,NaN,NaN,Great Britain,GBR,1896,Athina,GRE,15 April,1896-04-06 00:00:00,26.0
4,1896 Summer Olympics,1,GBR,Tennis,"Doubles, Men",John Pius Boland,79,Gold,John Pius Boland,Male,...,NaN,NaN,Great Britain,GBR,1896,Athina,GRE,15 April,1896-04-06 00:00:00,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252854,2020 Summer Olympics,61,USA,Taekwondo,"Featherweight, Women",Anastasija Zolotic,2506221,Gold,Anastasija Zolotic,Female,...,NaN,NaN,United States,USA,2020,Tokyo,JPN,8 August 2021,2021-07-23 20:20:00,19.0
252855,2020 Summer Olympics,61,UZB,Athletics,"Triple Jump, Women",Roksana Khudoyarova,2506229,NaN,Roksana Khudoyarova,Female,...,NaN,NaN,Uzbekistan,UZB,2020,Tokyo,JPN,8 August 2021,2021-07-23 20:20:00,20.0
252856,2020 Summer Olympics,61,UZB,Rhythmic Gymnastics,"Individual, Women",Yekaterina Fetisova,2506234,NaN,Yekaterina Fetisova,Female,...,NaN,NaN,Uzbekistan,UZB,2020,Tokyo,JPN,8 August 2021,2021-07-23 20:20:00,19.0
252857,2020 Summer Olympics,61,UZB,Swimming,"50 metres Freestyle, Women",Natalya Kritinina,2506239,NaN,Natalya Kritinina,Female,...,176.0,64,Uzbekistan,UZB,2020,Tokyo,JPN,8 August 2021,2021-07-23 20:20:00,21.0
